# Time reversal example

In [1]:
import pystablemotifs as sm
import pyboolnet.trap_spaces as ts

## Importing the network: TLGL-Small
This network is a simplified model of TLGL cancer. We will use time reversal to analyze a simplified TLGL model.

In [2]:
primes = sm.format.import_primes('../models/TLGL_Small.txt',remove_constants=True)
sm.format.pretty_print_prime_rules(primes)

apo* = disc | apo
cer* = !apo & fas & !s1p
disc* = !apo & fas & !flip | !apo & cer
fas* = !apo & !s1p
flip* = !apo & !disc
s1p* = !apo & !cer


## Computing the time reversal and garden spaces

In [3]:
trprimes=sm.time_reversal.time_reverse_primes(primes)
sm.format.pretty_print_prime_rules(trprimes)

apo* = apo & !disc
cer* = s1p | !fas | apo
disc* = !cer & flip | !cer & !fas | apo
fas* = s1p | apo
flip* = disc | apo
s1p* = cer | apo


The stable motifs (maximal trap spaces) of the time-reversed system are the garden spaces of the forward-time system.

In [4]:
gs=ts.trap_spaces(trprimes,'max')
for x in gs: print(x)

{'cer': 1, 's1p': 1}
{'apo': 0}


## Partitioning statespace and analyzing the partitions
Thus, the state-space is partitioned into four regions:

1. cer & s1p = 1, apo = 0
2. cer & s1p = 1, apo = 1
3. cer & s1p = 0, apo = 0
4. cer & s1p = 0, apo = 1

and no attractor lies in more than one region.
Note that these are NOT the garden spaces, which are instead defined by the variable `gs` above, i.e., they are regions 1, 1 union 2, and 1 union 3.

We will analyze these regions using the logical domain of influence (LDOI), which is describes the percolation of an input set.

Consider regions 1 and 2 and the LDOI of `{(cer,1),(s1p,1)}`:

In [5]:
imp12,con12 = sm.drivers.logical_domain_of_influence({'cer': 1, 's1p': 1},primes)
print('logical domain of influence:',imp12)
print('contradiction boundary:',con12)

logical domain of influence: {'fas': 0}
contradiction boundary: {'cer': 0, 's1p': 0}


The contradiction boundary is non-empty, and therefore, neither region 1 nor 2 contains any attractor.

Next, consider region 4 and the LDOI of `{(apo,1)}`:

In [6]:
impa1,cona1 = sm.drivers.logical_domain_of_influence({'apo': 1},primes)
print('logical domain of influence:',impa1)
print('contradiction boundary:',cona1)

logical domain of influence: {'apo': 1, 'cer': 0, 'disc': 0, 'fas': 0, 'flip': 0, 's1p': 0}
contradiction boundary: {}


Because the LDOI fixes all variable values, region 4 contains this fixed point and no other attractors.

Now, consider region 3, in which `apo=0`. The condition `cer & s1p = 0` can be satisfied for `s1p = 0` or for `cer = 0` (or both). We consider these two possibilities separately.

In [7]:
imps0,cons0 = sm.drivers.logical_domain_of_influence({'s1p': 0,'apo':0},primes)
impc0,conc0 = sm.drivers.logical_domain_of_influence({'cer': 0,'apo':0},primes)
print('logical domain of influence (s1p=0):\n',imps0)
print('contradiction boundary (s1p=0):\n',cons0)
print('\nlogical domain of influence (cer=0):\n',impc0)
print('contradiction boundary (cer=0):\n',conc0)

logical domain of influence (s1p=0):
 {'fas': 1, 'cer': 1, 'disc': 1, 'flip': 0, 's1p': 0}
contradiction boundary (s1p=0):
 {'apo': 1}

logical domain of influence (cer=0):
 {'s1p': 1, 'cer': 0, 'fas': 0, 'disc': 0, 'flip': 1, 'apo': 0}
contradiction boundary (cer=0):
 {}


Note that `{(s1p,0),(apo,0)}` has non-empty contradiction boundary. Therefore, region 3 cannot have an attractor with `s1p=0`.

The set `{(cer,0),(apo,0)}` has a complete LDOI, and therefore, region 3 contains the corresponding steady state.

Together, these statements imply that the steady state associated with the LDOI of `{(cer,0),(apo,0)}` is the only attractor in region 3.

Therefore, the attractors of the system are given as follows:

In [8]:
print(impa1)
print(impc0)

{'apo': 1, 'cer': 0, 'disc': 0, 'fas': 0, 'flip': 0, 's1p': 0}
{'s1p': 1, 'cer': 0, 'fas': 0, 'disc': 0, 'flip': 1, 'apo': 0}
